# Creating the Contabot Database

In order to create a integrated bot with a enterprise database, we will use some mock data available in [this repository](https://github.com/sinjoysaha/sales-analysis) (credits to [@sinjoysaha](https://github.com/sinjoysaha)) to simulate a history of sales.

## Steps
1. Clone the repository
2. Read the CSV files
3. Load all these files to a pandas dataframe
    a. Remove empty examples
4. Merge the dataframes
5. Save the examples to an unique dataframe
6. Save a new CSV file with all the examples

## Importing the libraries

In [163]:
import pandas as pd
import numpy as np
import os

## Settings

In [164]:
CURRENT_PATH = os.getcwd()
REPOSITORY_DATASET_PATH = os.path.join(CURRENT_PATH, 'sales-analysis', 'all_data.csv')
OUTPUT_ORDERS_PATH = os.path.join(CURRENT_PATH, 'output-orders.csv')
OUTPUT_PRODUCTS_PATH = os.path.join(CURRENT_PATH, 'output-products.csv')

## Importing the dataset

In [165]:
!git clone https://github.com/sinjoysaha/sales-analysis

fatal: destination path 'sales-analysis' already exists and is not an empty directory.


In [166]:
sales_dataset = pd.read_csv(REPOSITORY_DATASET_PATH, delimiter=',')

## Preprocessing

Removing incorrect rows

In [167]:
incorrect_samples_dataframe = sales_dataset[sales_dataset['Product'] == 'Product']
sales_dataset = sales_dataset.drop(incorrect_samples_dataframe.index)
sales_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 186495 entries, 0 to 186849
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Order ID          185950 non-null  object
 1   Product           185950 non-null  object
 2   Quantity Ordered  185950 non-null  object
 3   Price Each        185950 non-null  object
 4   Order Date        185950 non-null  object
 5   Purchase Address  185950 non-null  object
dtypes: object(6)
memory usage: 10.0+ MB


Removing empty rows

In [168]:
sales_dataset = sales_dataset.dropna()
sales_dataset = sales_dataset.drop(columns=['Order ID'])
sales_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 185950 entries, 0 to 186849
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   Product           185950 non-null  object
 1   Quantity Ordered  185950 non-null  object
 2   Price Each        185950 non-null  object
 3   Order Date        185950 non-null  object
 4   Purchase Address  185950 non-null  object
dtypes: object(5)
memory usage: 8.5+ MB


Creating a dataset just with the products

In [169]:
product_dataset = sales_dataset.drop_duplicates(subset=['Product'])
product_dataset = product_dataset.drop(columns=['Order Date', 'Purchase Address', 'Quantity Ordered'])
product_dataset = product_dataset.rename({'Price Each': 'Price'}, axis='columns')
product_dataset['ID'] = range(len(product_dataset))
product_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 19 entries, 0 to 109
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Product  19 non-null     object
 1   Price    19 non-null     object
 2   ID       19 non-null     int64 
dtypes: int64(1), object(2)
memory usage: 608.0+ bytes


Matching each order with its respective product 

In [170]:
product_dataset = product_dataset.set_index('Product', drop=False)
products_as_dict = product_dataset.to_dict('index')

def match_product_order(x):
    id = products_as_dict[x['Product']]
    x['Product ID'] = id['ID']
    return x

sales_dataset['Product ID'] = 0
sales_dataset = sales_dataset.apply(match_product_order, axis=1)

Removing remaining columns that will be not used 

In [172]:
sales_dataset = sales_dataset.drop(columns=['Product', 'Price Each', 'Purchase Address'])

## Saving the unified dataset

In [173]:
sales_dataset.to_csv(OUTPUT_ORDERS_PATH, sep=';', index=False)

In [174]:
product_dataset.to_csv(OUTPUT_PRODUCTS_PATH, sep=';', index=False)